In [46]:
import subprocess
import os
print('Current conda environment:', os.environ['CONDA_DEFAULT_ENV'])
os.environ['TOKENIZERS_PARALLELISM'] = "false"

cwd = os.getcwd()
print('Working directory:', cwd)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)

Current conda environment: reinvent
Working directory: /home/fts_g_ucla_edu/Projects/rips-relay/experiments


In [10]:
import pandas as pd

from rdkit import Chem

from molscore import MolScore
from moleval.metrics.metrics import GetMetrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, roc_auc_score, RocCurveDisplay

from xgboost import XGBClassifier

import pickle

In [11]:
fragments = []

f = open("data/fragments.smi", "r")

for i in range(100):
    mol = f.readline()
    fragments.append(mol[:-1])

In [12]:
fragments[:5]

['C1c2cncnc2NC1',
 'Cc1n[nH]c(N)c1C#N',
 'NCc1cc(Br)cnc1',
 'CC(Nc1nccs1)=O',
 'Cc1c(CO)cncc1']

### Generate a distribution from each fragment for each model

In [19]:
reinvent_distributions, crem_distributions, coati_distributions, safe_distributions = [], [], [], []
models = ['reinvent', 'crem', 'coati', 'safe']

In [26]:
%%capture

for fragment in fragments[:5]:
    for model in models:

        DF_FILEPATH = f'data/{model}_dataframe.csv'

        arg1 = '--model'
        arg2 = '--input_frag'
        arg3 = '--sample'

        args = ['python3', 'generate_analogs.py',
                arg1, model,
                arg2, fragment,
                arg3, '50']

        # Change directory to generate analogs with python script
        %cd ..

        subprocess.run(args,
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
                
        # Change directory back to that of the current notebook
        %cd experiments

        df = pd.read_csv(DF_FILEPATH, index_col=0)

        df['Model'] = model

        if model == 'reinvent':
            reinvent_distributions.append(df)
        elif model == 'crem':
            crem_distributions.append(df)
        elif model == 'coati':
            coati_distributions.append(df)
        elif model == 'safe':
            safe_distributions.append(df)

In [49]:
data = {'reinvent' : reinvent_distributions,
        'crem' : crem_distributions,
        'coati' : coati_distributions,
        'safe' : safe_distributions}

In [50]:
with open('lists.pkl', 'wb') as file:
    pickle.dump(data, file)

In [13]:
with open('lists.pkl', 'rb') as file:
    data = pickle.load(file)

In [14]:
reinvent_distributions = data['reinvent']
crem_distributions = data['crem']
coati_distributions = data['coati']
safe_distributions = data['safe']

In [15]:
smiles = reinvent_distributions[0]['SMILES'].to_list()

In [16]:
smiles

['O=[N+]([O-])c1ccc2c(n1)C(O)CN2',
 'CC(C)Oc1ncnc2c1CNC2',
 'Nc1nc2c(nc1Br)CCN2',
 'Nc1nc(Cl)c2c(n1)CNCC2',
 'Cc1nc(N)c2c(n1)CCN2C',
 'Cc1nc2c(nc1Cl)NCC2',
 'CN1CCc2nc(N)c3ncc(N)nc3c2C1',
 'Brc1ccc2[nH]ncc2n1',
 'CC(C)[C@H]1Cc2cncnc2N1',
 'CC1CNc2ncncc2C1',
 'CCC1Cc2cncnc2N1',
 'Nc1cc2c(cn1)CNCC2',
 'CCCNc1nc2c(nc1O)CCN2',
 'c1ncc2c(n1)NC1(CC2)CC1',
 'CC1CNc2cncnc21',
 'c1ncc2c(n1)NC1CCCCC21',
 'O=[N+]([O-])c1cc2c(cn1)C(O)CN2',
 'CN1CCc2cncnc21',
 'c1c[nH]c(-c2ncc3c(n2)NCC3)c1',
 'CC1CNc2ncncc2N1',
 'N#Cc1ncc2c(n1)NCC2',
 'CC1Cc2nc(N)cnc2N1',
 'CC1CCCNc2ncncc21',
 'Nc1nc(N)c2c(n1)CCN2',
 'CC1Cc2nc(N)nc(N)c2N1',
 'O=[N+]([O-])c1cc2c(cn1)CNCC2',
 'CNc1nc2c(nc1C)NCC2',
 'C[C@@H]1Cc2cnc(N)nc2N1',
 'Brc1ncnc2c1CCN2',
 'CC1CCNc2nccnc21',
 'Clc1ccc2[nH]cnc2n1',
 'c1ncc2c(n1)OCCCC2',
 'CCN1CCc2cncnc21',
 'Brc1ncc2c(n1)CCN2',
 'c1cnc2c(c1)CCN2',
 'CC1(C)CNc2ncncc21',
 'COC1CNc2nccnc2CN1',
 'c1cnc2c(n1)CCN2',
 'C[C@@H]1CNc2cncnc2N1',
 'c1ncc2c(n1)OCCC2',
 'O=[N+]([O-])c1ccc2[nH]ncc2c1',
 'c1ccc(

In [21]:
smiles2 = crem_distributions[1]['SMILES'].to_list()
smiles3 = coati_distributions[0]['SMILES'].to_list()
smiles4 = safe_distributions[0]['SMILES'].to_list()

In [17]:
MetricEngine = GetMetrics(
    n_jobs=1,
    device='cpu',
    batch_size=512,
    run_fcd=False
)
metrics = MetricEngine.calculate(
    smiles,
    calc_valid=True,
    calc_unique=True,
    unique_k=10000,
    se_k=1000,
    sp_k=1000,
    properties=True,
)

Cleaning up reference smiles


/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:205: UserWarning: Less than 1000 molecules so SEDiv is non-standard.
  warnings.warn(f'Less than {se_k} molecules so SEDiv is non-standard.')
/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:211: UserWarning: Less than 1000 molecules so SPDiv is non-standard.
  warnings.warn(f'Less than {sp_k} molecules so SPDiv is non-standard.')


In [18]:
metrics

{'#': 50,
 'Validity': 1.0,
 '# valid': 50,
 'Uniqueness': 0.98,
 '# valid & unique': 49,
 'IntDiv1': 0.7948251545106506,
 'IntDiv2': 0.7476713129919497,
 'SEDiv': 0.46938775510204084,
 'SPDiv': 0.9840981234875023,
 '# scaffolds': 28,
 'ScaffDiv': 0.697516663342106,
 'ScaffUniqueness': 0.5714285714285714,
 'FG': 0.08743169398907104,
 'RS': 0.5490196078431373,
 'Filters': 0.5918367346938775,
 'Purchasable_ZINC20': 0.08163265306122448}

In [22]:
MetricEngine = GetMetrics(
    n_jobs=1,
    device='cpu',
    batch_size=512,
    run_fcd=False
)
metrics2 = MetricEngine.calculate(
    smiles2,
    calc_valid=True,
    calc_unique=True,
    unique_k=10000,
    se_k=1000,
    sp_k=1000,
    properties=True,
)

Cleaning up reference smiles


/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:205: UserWarning: Less than 1000 molecules so SEDiv is non-standard.
  warnings.warn(f'Less than {se_k} molecules so SEDiv is non-standard.')
/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:211: UserWarning: Less than 1000 molecules so SPDiv is non-standard.
  warnings.warn(f'Less than {sp_k} molecules so SPDiv is non-standard.')


In [23]:
metrics2

{'#': 50,
 'Validity': 1.0,
 '# valid': 50,
 'Uniqueness': 1.0,
 '# valid & unique': 50,
 'IntDiv1': 0.8960155154228211,
 'IntDiv2': 0.8251595270036898,
 'SEDiv': 0.94,
 'SPDiv': 0.9949364419468035,
 '# scaffolds': 8,
 'ScaffDiv': 0.6374329067766666,
 'ScaffUniqueness': 0.16,
 'FG': 0.4722222222222222,
 'RS': 0.7666666666666667,
 'Filters': 0.26,
 'Purchasable_ZINC20': 0.48}

In [26]:
MetricEngine = GetMetrics(
    n_jobs=1,
    device='cpu',
    batch_size=512,
    run_fcd=False
)
metrics3 = MetricEngine.calculate(
    smiles3,
    calc_valid=True,
    calc_unique=True,
    unique_k=10000,
    se_k=1000,
    sp_k=1000,
    properties=True,
)

Cleaning up reference smiles


/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:205: UserWarning: Less than 1000 molecules so SEDiv is non-standard.
  warnings.warn(f'Less than {se_k} molecules so SEDiv is non-standard.')
/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:211: UserWarning: Less than 1000 molecules so SPDiv is non-standard.
  warnings.warn(f'Less than {sp_k} molecules so SPDiv is non-standard.')


In [27]:
metrics3

{'#': 50,
 'Validity': 1.0,
 '# valid': 50,
 'Uniqueness': 1.0,
 '# valid & unique': 50,
 'IntDiv1': 0.7771809272766114,
 'IntDiv2': 0.7398999224879415,
 'SEDiv': 0.6,
 'SPDiv': 0.9868649318951688,
 '# scaffolds': 49,
 'ScaffDiv': 0.7617993031080342,
 'ScaffUniqueness': 0.98,
 'FG': 0.15151515151515152,
 'RS': 0.5675675675675675,
 'Filters': 0.54,
 'Purchasable_ZINC20': 0.22}

In [28]:
MetricEngine = GetMetrics(
    n_jobs=1,
    device='cpu',
    batch_size=512,
    run_fcd=False
)
metrics4 = MetricEngine.calculate(
    smiles4,
    calc_valid=True,
    calc_unique=True,
    unique_k=10000,
    se_k=1000,
    sp_k=1000,
    properties=True,
)

Cleaning up reference smiles


/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:205: UserWarning: Less than 1000 molecules so SEDiv is non-standard.
  warnings.warn(f'Less than {se_k} molecules so SEDiv is non-standard.')
/home/fts_g_ucla_edu/.local/lib/python3.10/site-packages/moleval/metrics/metrics.py:211: UserWarning: Less than 1000 molecules so SPDiv is non-standard.
  warnings.warn(f'Less than {sp_k} molecules so SPDiv is non-standard.')


In [29]:
metrics4

{'#': 50,
 'Validity': 1.0,
 '# valid': 50,
 'Uniqueness': 1.0,
 '# valid & unique': 50,
 'IntDiv1': 0.8577182090759278,
 'IntDiv2': 0.8078390865298755,
 'SEDiv': 0.98,
 'SPDiv': 0.9941221599368881,
 '# scaffolds': 45,
 'ScaffDiv': 0.8265063596654821,
 'ScaffUniqueness': 0.9,
 'FG': 0.32142857142857145,
 'RS': 0.31666666666666665,
 'Filters': 0.82,
 'Purchasable_ZINC20': 0.0}